In [11]:
import pandas as pd

In [ ]:
def read_data(data_path):
    "load patient surgical data"
    df = pd.read_excel(data_path)
    
    # Clean column names: remove encoding artifacts and excessive whitespace
    df.columns = (df.columns
                  .astype(str)
                  .str.replace(r'_x[0-9A-F]{4}_', '', regex=True)  # Remove _xHHHH_ patterns
                  .str.replace(r'_x[0-9A-F]{4}$', '', regex=True)  # Remove _xHHHH_ at end
                  .str.replace(r'\s+', ' ', regex=True)  # Collapse multiple spaces
                  .str.strip())  # Remove leading/trailing whitespace
    
    return df

def clean_data(df):
    "clean patient surgical data"

    #unify spelling
    df["UIV_implant"] = df["UIV_implant"].str.capitalize().replace({"Fs": "FS", "Ps": "PS"})
    df.loc[df["UIV_implant"].str.contains("Fenestrated", na=False), "UIV_implant"] = "FS"
    df.loc[df["UIV_implant"].str.contains("ether", na=False), "UIV_implant"] = "PS"
    df["sex"]= df["sex"].replace({"F": "FEMALE", "M": "MALE"})

    #create column with updated num_levels
    df["num_levels"] = pd.to_numeric(df["num_levels"], errors="coerce").astype("Int64")
    df.loc[df["num_levels"] >= 10, "num_levels_cat"] = "higher"
    df.loc[df["num_levels"] < 10, "num_levels_cat"] = "lower"

    #rename global_tilt column
    df.rename(columns={"global_tilt...18":"global_tilt_preop"},inplace=True)
    df.rename(columns={"global_tilt...50":"global_tilt_postop"},inplace=True)
    df.rename(columns={"num_interbody_fusion_levels":"num_interbody_fusion_levels_this_time"},inplace=True)

    # encoding artifacts should already be cleaned by read_data
    # but add fallback for any remaining issues
    if "SVA_postop" not in df.columns and "SVA_postop_x000D_" in df.columns:
        df.rename(columns={"SVA_postop_x000D_":"SVA_postop"},inplace=True)

    #rename smoking
    df.rename(columns={"SMOKING": "smoking"}, inplace=True)

    df["smoking"] = df["smoking"].map({
    "Never smoker": 0,
    "Former smoker": 1,
    "Current some day smoker": 1,
    "Current every day smoker": 1
})

    # Revision indicator: interbody levels > 0 but no ALIF/XLIF/TLIF
    df["revision"] = (
        (df["num_interbody_fusion_levels"] > 0) &
        (df["ALIF"] == 0) &
        (df["XLIF"] == 0) &
        (df["TLIF"] == 0)
    ).astype(int)


    return df


In [13]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0]   # goes from notebooks/ → repo root
DATA_PATH = PROJECT_ROOT / "data" / "raw" / "MSDS_cleaned_with_CCI_ODI.xlsx"

df = read_data(DATA_PATH)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Columns: 107 entries, id to Prior_surgery
dtypes: float64(59), int64(21), object(27)
memory usage: 230.0+ KB


In [14]:
print(df["SMOKING"].value_counts(dropna=False))

SMOKING
Never smoker                108
Former smoker               103
NaN                          48
Current every day smoker     14
Current some day smoker       2
Name: count, dtype: int64


In [15]:
clean_df = clean_data(df)
#print(df.head())
print(clean_df[["num_levels", "num_levels_cat"]])

     num_levels num_levels_cat
0             9          lower
1            15         higher
2             9          lower
3            15         higher
4             9          lower
..          ...            ...
270          15         higher
271          15         higher
272          15         higher
273           9          lower
274          15         higher

[275 rows x 2 columns]


In [16]:
print(df["smoking"].value_counts(dropna=False))


smoking
1.0    119
0.0    108
NaN     48
Name: count, dtype: int64


The SMOKING variable was originally recorded as a categorical field (Never smoker, Former smoker, Current some day smoker, Current every day smoker). For modeling purposes, it was converted into a binary predictor where 0 represents never smokers and 1 represents former or current smokers. Approximately 17% of observations had missing smoking values; to preserve sample size and maintain consistency in the mechanical failure model, missing values were imputed as non-smoker (0).

In [17]:
df["smoking"] = df["smoking"].fillna(0)


In [18]:
print(df["smoking"].value_counts())


smoking
0.0    156
1.0    119
Name: count, dtype: int64


In [19]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0] 
processed_dir = PROJECT_ROOT / "data" / "processed"
processed_dir.mkdir(parents=True, exist_ok=True)

# Hold out test patients (not used for training)
HOLDOUT_IDS = [1176294, 2964021, 818588, 6380632]

PATIENT_DESCRIPTIONS = {
    1176294: "mech failure - large GAP improvement;",
    2964021: "revision - GAP improvement 3 to 1",
    818588: "mech failure - small GAP improvement;",
    6380632: "large GAP imporovement",
}

holdout_mask = clean_df["id"].isin(HOLDOUT_IDS)

holdout_df = clean_df[holdout_mask].copy()
train_df = clean_df[~holdout_mask]

# Add description column to holdout
holdout_df["description"] = holdout_df["id"].map(PATIENT_DESCRIPTIONS)

print(f"Total patients: {len(clean_df)}")
print(f"Holdout patients: {len(holdout_df)} (IDs: {HOLDOUT_IDS})")
print(f"Training patients: {len(train_df)}")

holdout_df.to_csv(processed_dir / "holdout_patients.csv", index=False)
train_df.to_csv(processed_dir / "cleaned_for_modeling.csv", index=False)

Total patients: 275
Holdout patients: 4 (IDs: [1176294, 2964021, 818588, 6380632])
Training patients: 271


In [20]:
postop_cols = [col for col in clean_df.columns if "postop" in col.lower()]
print(f"Postoperative columns ({len(postop_cols)}):")
for col in postop_cols:
    print(f"  {col}")


Postoperative columns (38):
  C7CSVL_postop
  SVA_postop
  TK_T4_T12_postop
  TK_T10_L2_postop
  PJA_postop
  T4PA_postop
  L1PA_postop
  L1PA_ideal_mismatch_postop
  T4L1PA_ideal_mismatch_postop
  global_tilt_postop
  LL_postop
  L4_S1_postop
  L5_S1_postop
  PT_postop
  PI_postop
  PI_LL_postop
  SS_postop
  cobb_prox_curve_postop
  EV_prox_prox_postop
  EV_distal_prox_postop
  cobb_main_curve_postop
  EV_prox_main_postop
  EV_distal_main_postop
  cobb_distal_curve_postop
  EV_prox_distal_postop
  EV_distal_distal_postop
  FC_postop
  RPV_postop
  RPV_score_postop
  RLL_postop
  RLL_score_postop
  LDI_postop
  LDI_score_postop
  RSA_postop
  RSA_score_postop
  gap_age_point_postop
  gap_score_postop
  gap_category_postop
